# 605.629: Programming Languages
## Assignment 6

Sabbir Ahmed

1. \[20 pts, Scheme\]

Write a function count so that `(count a lat)` counts how often the atom `a` appears in the list of atoms `lat`.

```
> (count 'a '(d a t a))
2
> (count 's '(m i s s i s s i p p i))
4
```

__Answer__

```scheme
(define (count a lat)
    (if (null? lat)                     ; if list is empty
        0                               ; return 0
        (if (equal? a (car lat))        ; else if a equals the first value of lat
            (+ 1 (count a (cdr lat)))   ; recurse function with remainder of lat and add 1
            (count a (cdr lat)))))
```

-----

2. \[20 pts, Scheme\]

Write a function `(count-all lat)` that produces a list pairing every individual atom in the list of atoms `lat` with a count of how often the atom appears in `lat`.

```
> (count-all '(m i s s i s s i p p i))
((m 1) (i 4) (s 4) (p 2))
> (count-all '(to be or not to be that is the question))
((to 2) (be 2) (or 1) (not 1) (that 1) (is 1) (the 1) (question 1))
```
The order of the items in the output list does not matter. Make sure to include the code if you use any auxiliary functions (helper functions). Note that this function is computing a histogram.

__Answer__

```scheme
(define (exclude lat a)
    (fold (lambda (alist result)    ; fold lambda into '() lat
        (if (equal? (car alist) a)  ; if a equals the first value of the association list
            result                  ; return the result of the lambda
            (cons alist result)))   ; else append cons pair of the association list and the result 
    '() lat))

(define (count-all lat)
    (fold (lambda (freq bin)
        (cond 
            ((assoc freq bin) => (lambda (old)
                (let ((new (cons freq (+ 1 (cdr old)))))
                    (cons new (exclude bin freq)))))
            (else
                (let ((new (cons freq 1)))
                    (cons new bin)))))
    '() lat))
```

-----

3. \[20 pts, Scheme\]

Consider the following code:

```scheme
(define mystery
    (lambda (x)
        (cond
             ((null? x) (list x))
            (else (append
                (mystery (cdr x))
                (map (lambda (y)
                    (cons (car x) y))
                    (mystery (cdr x))))))))
```

a. What is the value of `(mystery '(a b c))`?

__Answer__

Breaking down the function:

```scheme
(define mystery
    (lambda (x)
        (cond
            ((null? x) (list x))            ; if nil, return empty list
            (else (append                   ; else append the following
                (mystery (cdr x))           ; return value of the recursive call with the rest of x
                (map (lambda (y)            ; map the lambda with the recursive call
                    (cons (car x) y))       ; make a pair of the first value of x and y
                    (mystery (cdr x))))))))
```

The function appears to create lists with all the possible combinations of the elements of the input list. The following is generated from using an interpreter:

`(() (c) (b) (b c) (a) (a c) (a b) (a b c))`

----

b. Suggest a better name for mystery that better conveys its purpose.

__Answer__

The function appears to create a power set from the input list. `mystery` can be renamed to `power-set`.

----

c. What is the value of `(length (mystery '(a b c d e f g h i j)))`?

__Answer__

To find the number of power sets possible for a set of $n$ elements, compute $2^n$. Since `(length '(a b c d e f g h i j)) == 10`, then `(length (mystery '(a b c d e f g h i j)))` = $2^{10}=1024$.

----